I am going to attempt to develop a model that will predict who will win any individaul mens ncaa college basketball game. Then, I would like to see how accuratly the model can predict Michigan States record from the 2021 season. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

I got the data from https://barttorvik.com/trank.php?year=2021&sort=&top=0&conlimit=All#

In [8]:
data = pd.read_csv("cbb21.csv")

In [9]:
data

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,SEED
0,Michigan,B10,24,20,118.1,91.1,0.9521,54.9,44.9,16.3,...,24.8,28.9,24.5,53.3,42.3,38.7,33.5,66.9,7.2,1.0
1,Baylor,B12,24,22,123.2,94.5,0.9548,57.5,49.1,17.6,...,30.9,27.0,31.7,54.1,48.1,41.8,34.0,68.8,6.6,1.0
2,Illinois,B10,29,23,117.7,90.4,0.9539,55.6,46.6,18.2,...,22.2,39.2,30.5,55.3,45.4,37.6,32.7,70.7,8.9,1.0
3,Gonzaga,WCC,26,26,125.4,89.8,0.9791,61.0,47.5,16.1,...,23.4,36.7,25.9,64.0,46.8,36.5,32.5,74.6,8.5,1.0
4,Iowa,B10,29,21,123.5,95.7,0.9491,54.6,48.3,13.3,...,28.6,32.0,22.6,52.4,45.8,38.6,34.8,70.0,5.6,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,Louisiana Tech,CUSA,27,21,102.7,93.4,0.7479,50.5,45.6,18.4,...,23.4,35.3,26.4,49.7,46.4,34.6,29.6,69.6,-1.7,NaN
343,Toledo,MAC,29,21,113.3,101.8,0.7743,54.3,48.3,15.5,...,30.7,28.5,23.4,52.5,51.0,37.7,29.5,69.1,-2.1,NaN
344,UAB,CUSA,27,22,102.5,94.6,0.7153,48.6,47.0,15.6,...,25.4,29.9,27.1,49.4,46.1,31.1,32.4,67.5,-2.7,NaN
345,Eastern Kentucky,OVC,27,22,101.5,102.3,0.4749,51.2,51.0,16.6,...,30.2,25.6,32.2,49.6,51.8,35.8,33.0,75.1,-3.8,NaN


In [10]:
data = data.drop_duplicates(subset = "TEAM", keep="first")

I wanted to remove NA values, but realized if I were to do that, it would remove all of the teams who did not make the NCAA tournament at the end of the year. Instead, I just filled those values with a string for "missed tournament"

In [12]:
data.fillna({'SEED':'Missed Tournament'}, inplace = True)
data.SEED.value_counts(dropna=False)

Missed Tournament    279
16.0                   6
11.0                   6
10.0                   4
15.0                   4
14.0                   4
13.0                   4
12.0                   4
1.0                    4
2.0                    4
8.0                    4
7.0                    4
6.0                    4
5.0                    4
4.0                    4
3.0                    4
9.0                    4
Name: SEED, dtype: int64

In [13]:
data

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,SEED
0,Michigan,B10,24,20,118.1,91.1,0.9521,54.9,44.9,16.3,...,24.8,28.9,24.5,53.3,42.3,38.7,33.5,66.9,7.2,1.0
1,Baylor,B12,24,22,123.2,94.5,0.9548,57.5,49.1,17.6,...,30.9,27.0,31.7,54.1,48.1,41.8,34.0,68.8,6.6,1.0
2,Illinois,B10,29,23,117.7,90.4,0.9539,55.6,46.6,18.2,...,22.2,39.2,30.5,55.3,45.4,37.6,32.7,70.7,8.9,1.0
3,Gonzaga,WCC,26,26,125.4,89.8,0.9791,61.0,47.5,16.1,...,23.4,36.7,25.9,64.0,46.8,36.5,32.5,74.6,8.5,1.0
4,Iowa,B10,29,21,123.5,95.7,0.9491,54.6,48.3,13.3,...,28.6,32.0,22.6,52.4,45.8,38.6,34.8,70.0,5.6,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,Louisiana Tech,CUSA,27,21,102.7,93.4,0.7479,50.5,45.6,18.4,...,23.4,35.3,26.4,49.7,46.4,34.6,29.6,69.6,-1.7,Missed Tournament
343,Toledo,MAC,29,21,113.3,101.8,0.7743,54.3,48.3,15.5,...,30.7,28.5,23.4,52.5,51.0,37.7,29.5,69.1,-2.1,Missed Tournament
344,UAB,CUSA,27,22,102.5,94.6,0.7153,48.6,47.0,15.6,...,25.4,29.9,27.1,49.4,46.1,31.1,32.4,67.5,-2.7,Missed Tournament
345,Eastern Kentucky,OVC,27,22,101.5,102.3,0.4749,51.2,51.0,16.6,...,30.2,25.6,32.2,49.6,51.8,35.8,33.0,75.1,-3.8,Missed Tournament
